# Logistic Regression Including REFINED Variables

### We will use the features from the XGBoost refined features model 
### This ensures consistency, reduces complexity, and leverages the feature selection process that has already been validated with my refined features XGBoost model
### This would also allow for a fair comparison between models

In [1]:
import pandas as pd
composite_preprocessed = pd.read_csv('Composite_preprocessed_NO_MV_BALANCED.csv')
composite_preprocessed.head()

,Number of Stars,Number of Planets,Number of Moons,Galactic Latitude [deg],Galactic Longitude [deg],Ecliptic Latitude [deg],Ecliptic Longitude [deg],Number of Photometry Time Series,Number of Radial Velocity Time Series,Number of Stellar Spectra Measurements,Number of Emission Spectroscopy Measurements,Number of Transmission Spectroscopy Measurements,Circumbinary Flag,Controversial Flag,Discovery Year,Detected by Transits
0,3.094076,-0.666894,0.0,2.424559,1.234306,-0.145901,-0.624689,0.613405,1.731519,-0.261600,-0.012466,-0.044364,0,0,2007,0
1,-0.256668,-0.666894,0.0,1.186720,-0.283545,1.148661,-0.992906,0.613405,0.729626,-0.261600,-0.012466,-0.044364,0,0,2009,0
2,-0.256668,-0.666894,0.0,-0.877523,-0.306068,0.308947,-2.327878,0.613405,0.729626,-0.261600,-0.012466,-0.044364,0,0,2008,0
3,-0.256668,0.216988,0.0,1.382856,-0.669803,0.872499,-0.152934,0.613405,3.735304,0.591749,-0.012466,-0.044364,0,0,2002,0
4,6.444820,-0.666894,0.0,0.261241,-0.531444,1.023143,0.855489,0.613405,3.735304,2.298449,-0.012466,-0.044364,0,0,1996,0


# Drop Unnecessary

In [2]:
remove = ['Number of Radial Velocity Time Series',
          'Number of Stellar Spectra Measurements',
          'Controversial Flag',
          'Circumbinary Flag',
          'Number of Moons']
composite_refined = composite_preprocessed.drop(remove, axis=1)
composite_refined.head()

,Number of Stars,Number of Planets,Galactic Latitude [deg],Galactic Longitude [deg],Ecliptic Latitude [deg],Ecliptic Longitude [deg],Number of Photometry Time Series,Number of Emission Spectroscopy Measurements,Number of Transmission Spectroscopy Measurements,Discovery Year,Detected by Transits
0,3.094076,-0.666894,2.424559,1.234306,-0.145901,-0.624689,0.613405,-0.012466,-0.044364,2007,0
1,-0.256668,-0.666894,1.186720,-0.283545,1.148661,-0.992906,0.613405,-0.012466,-0.044364,2009,0
2,-0.256668,-0.666894,-0.877523,-0.306068,0.308947,-2.327878,0.613405,-0.012466,-0.044364,2008,0
3,-0.256668,0.216988,1.382856,-0.669803,0.872499,-0.152934,0.613405,-0.012466,-0.044364,2002,0
4,6.444820,-0.666894,0.261241,-0.531444,1.023143,0.855489,0.613405,-0.012466,-0.044364,1996,0


# Examine VIF

## Although we were able to filter out some variables from the model, we should still check VIF (Variance Inflation Factor) to ensure there is no multicollinearity in these remaining variables

## VIF measures how much the variance of a regression coefficient is inflated due to multicollinearity with other predictors
### A VIF value greater than 5 indicates high multicollinearity

In [3]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

# we are trying to predict whether an exoplanet has been detected by transits
targets = composite_refined['Detected by Transits']
# training features are all variables except the targets
features = composite_refined.drop(['Detected by Transits'], axis=1)

### Add constant for VIF calculation

In [4]:
features_with_intercept = add_constant(features)

### Calculate VIF

In [5]:
vif_data = pd.DataFrame()
vif_data['Feature'] = features_with_intercept.columns
vif_data['VIF'] = [variance_inflation_factor(features_with_intercept.values, i) for i in range(features_with_intercept.shape[1])]
vif_data

,Feature,VIF
0,const,183017.944260
1,Number of Stars,1.042952
2,Number of Planets,1.038336
3,Galactic Latitude [deg],1.576786
4,Galactic Longitude [deg],2.207166
5,Ecliptic Latitude [deg],3.087281
6,Ecliptic Longitude [deg],1.356383
7,Number of Photometry Time Series,1.120718
8,Number of Emission Spectroscopy Measurements,1.004541
9,Number of Transmission Spectroscopy Measurements,1.007780


### VIF values of 4-5 are moderate. None of the features demonstrate moderate to high VIF

# Train Test Split

## Observe Feature Correlations

In [6]:
features_final=features
features_final.corr()

,Number of Stars,Number of Planets,Galactic Latitude [deg],Galactic Longitude [deg],Ecliptic Latitude [deg],Ecliptic Longitude [deg],Number of Photometry Time Series,Number of Emission Spectroscopy Measurements,Number of Transmission Spectroscopy Measurements,Discovery Year
Number of Stars,1.000000,0.109074,-0.044988,0.104236,-0.087528,-0.069450,0.057734,-0.003200,-0.002096,-0.147575
Number of Planets,0.109074,1.000000,-0.055511,0.008589,0.026680,-0.023926,-0.002505,0.057797,0.076409,-0.072274
Galactic Latitude [deg],-0.044988,-0.055511,1.000000,-0.019213,0.463063,0.174632,-0.090369,-0.025691,-0.008918,0.031820
Galactic Longitude [deg],0.104236,0.008589,-0.019213,1.000000,-0.657044,-0.408779,0.131333,-0.008283,-0.026088,-0.106558
Ecliptic Latitude [deg],-0.087528,0.026680,0.463063,-0.657044,1.000000,0.498442,-0.222283,-0.006864,0.025212,0.081113
Ecliptic Longitude [deg],-0.069450,-0.023926,0.174632,-0.408779,0.498442,1.000000,-0.119612,0.013811,0.031078,0.058041
Number of Photometry Time Series,0.057734,-0.002505,-0.090369,0.131333,-0.222283,-0.119612,1.000000,-0.004172,-0.014848,-0.252983
Number of Emission Spectroscopy Measurements,-0.003200,0.057797,-0.025691,-0.008283,-0.006864,0.013811,-0.004172,1.000000,-0.000553,0.000791
Number of Transmission Spectroscopy Measurements,-0.002096,0.076409,-0.008918,-0.026088,0.025212,0.031078,-0.014848,-0.000553,1.000000,-0.015425
Discovery Year,-0.147575,-0.072274,0.031820,-0.106558,0.081113,0.058041,-0.252983,0.000791,-0.015425,1.000000


## Split data

In [7]:
# Splitting dataset into training and testing addresses overfitting
# shuffling is necessary to remove dependencies that come from order of data
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(features_final, targets, train_size = 0.8, random_state = 42)

x_train.shape, y_train.shape

((5148, 10), (5148,))

# Fitting the Logistic Regression

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

## Use L1 regularization (Lasso) for:
### Enhanced feature selection (shrinks some features to exactly 0, simplifying our model with only the most important features
### Handles multicollinearity ; penalizes large coefficients and encourages a sparse model
### Prevents overfitting ; Penalty for large coefficients reduces overfitting especially when number features is large compared to num obs

## Use liblinear solver because:
### Suitable for small-medium sized datasets (uses coordinate descent algorithm, efficient for L1 regularization
### Supports L1 regularization (for logreg with L1 penalty)
### Works great with binary classification problems

## Max iterations: 1000
### This is to ensure our optimization algorithm has enough iterations to reach convergence 
### Logregs especially with regularization require higher number of iterations to find optimal coefficients 
### Avoids premature stopping 

In [9]:
logreg = LogisticRegression(penalty='l1', solver='liblinear', max_iter=1000)
logreg.fit(x_train,y_train)

LogisticRegression(max_iter=1000, penalty='l1', solver='liblinear')

In [10]:
logreg.score(x_train, y_train)

0.8560606060606061

### 85% of the model's training outputs match the targets!
### Still important to manually calculate the accuracy

In [11]:
model_outputs = logreg.predict(x_train)
# view the predicted class labels of the regression
model_outputs

array([0, 1, 1, ..., 1, 0, 0])

In [12]:
model_outputs == y_train

915      True
5904    False
2083     True
2480     True
509      True
        ...  
3772     True
5191     True
5226    False
5390     True
860      True
Name: Detected by Transits, Length: 5148, dtype: bool

In [13]:
# sum to measure correct predictions
import numpy as np
# number correct / total number model predictions
np.sum([model_outputs == y_train]) / model_outputs.shape[0]

0.8560606060606061

### Same accuracy!

# Analyze Summary Table with coefficients (weights) and intercept (bias)

In [14]:
logreg.intercept_, logreg.coef_

(array([0.]),
 array([[-2.49614918e-01,  4.85550686e-01, -1.91117595e-01,
          4.07527467e-01,  1.77649881e+00,  4.34578143e-01,
         -4.70618828e+00,  2.24005499e-02,  3.78128071e-01,
         -4.57715140e-04]]))

### Match up coefs with features

In [15]:
feature_name = features_final.columns.values
summary_table = pd.DataFrame(columns=['Feature Name'], data = feature_name)

### Transpose bc by default np arrays are rows and not columns

In [16]:
summary_table['Coefficients'] = np.transpose(logreg.coef_)

### Insert intercept as 0th index ; Move all indices up by 1 

In [17]:
summary_table.index += 1
summary_table.loc[0] = ['Intercept', logreg.intercept_[0]]
summary_table = summary_table.sort_index()
summary_table

,Feature Name,Coefficients
0,Intercept,0.000000
1,Number of Stars,-0.249615
2,Number of Planets,0.485551
3,Galactic Latitude [deg],-0.191118
4,Galactic Longitude [deg],0.407527
5,Ecliptic Latitude [deg],1.776499
6,Ecliptic Longitude [deg],0.434578
7,Number of Photometry Time Series,-4.706188
8,Number of Emission Spectroscopy Measurements,0.022401
9,Number of Transmission Spectroscopy Measurements,0.378128


## The closer the coefficient is to 0, the less impact (less weight on the model) 
## Now we calculate log odds 
## Log odds tell us the strength of the relationship between the feature and the outcome (detected by transits T/F)

In [18]:
summary_table['Odds_ratio'] = np.exp(summary_table['Coefficients'])
summary_ordered = summary_table.sort_values('Odds_ratio', ascending=False)
summary_ordered

,Feature Name,Coefficients,Odds_ratio
5,Ecliptic Latitude [deg],1.776499,5.909131
2,Number of Planets,0.485551,1.625070
6,Ecliptic Longitude [deg],0.434578,1.544311
4,Galactic Longitude [deg],0.407527,1.503097
9,Number of Transmission Spectroscopy Measurements,0.378128,1.459550
8,Number of Emission Spectroscopy Measurements,0.022401,1.022653
0,Intercept,0.000000,1.000000
10,Discovery Year,-0.000458,0.999542
3,Galactic Latitude [deg],-0.191118,0.826035
1,Number of Stars,-0.249615,0.779101


## A feature is not important if:
## Coef ~ 0 (means this coefficient, even at its highest value will have close to 0 effect on the outcome);
## Odds ratio ~ 1 (means this coefficient, even at its highest value will only have minimal effect on the odds of the outcome)

# Calculating P-values

### Using Logistic Regression from statsmodels for more in-depth statistical analysis and interpretation of the model coefficients

In [19]:
import statsmodels.api as sm

# add constant to prevent singular matrix error
x_train_with_intercept = sm.add_constant(x_train)

# Train (fit) the model and print summary
# Use regularization technique L1 Lasso to address quasi-separation and prevents coefficients from becoming too large
sm_logreg = sm.Logit(y_train, x_train_with_intercept).fit_regularized(method='l1',maxiter=1000)
sm_logreg.summary()

/Users/kevin/anaconda3/envs/py310_ds/lib/python3.10/site-packages/statsmodels/discrete/discrete_model.py:2383: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/Users/kevin/anaconda3/envs/py310_ds/lib/python3.10/site-packages/statsmodels/discrete/discrete_model.py:2441: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q * linpred)))


Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.3119125439719923
            Iterations: 66
            Function evaluations: 72
            Gradient evaluations: 66


LinAlgError: Singular matrix

# ROC-AUC Score

In [ ]:
# predict on the test set
y_pred = logreg.predict(x_test)
y_prob = logreg.predict_proba(x_test)[:,1]
roc_auc = metrics.roc_auc_score(y_test, y_prob)
roc_auc

# Confusion Matrix

In [ ]:
import matplotlib.pyplot as plt
conf_matrix = metrics.confusion_matrix(y_test, y_pred)
metrics.ConfusionMatrixDisplay(conf_matrix).plot()
plt.show()

# Cross validation
### Ensuress model's performance is robust and not overly dependent on a single test-train split

In [ ]:
from sklearn.model_selection import cross_val_score
# specify 5-fold cv
# splits data into 5 parts; training model on 4 and validating on 5th
# repeats process 5 times
# end result; an average performance metric that provides better estimate of the model's performance 
cv_score = cross_val_score(logreg, x_train, y_train, cv=5) 
print('cv score:', cv_score)
print('mean cv score:', np.mean(cv_score))

# Hyperparameter Tuning
### Use gridsearch to find optimal hyperparameters for the model
### penalty: regularization l1 or l2
### C: list of possible values for the inverse of regularization strength 
#### Smaller values of C imply stronger regularization (to prevent overfitting), while larger values imply weaker regularization (allowing the model to fit the training data more closely)
### solver: algorithm to use in the optimization problem (liblinear is suitable for small datasets and supports l1 and l2 regularization)

In [ ]:
from sklearn.model_selection import GridSearchCV

parameter_grid = {
    'penalty' : ['l1','l2'],
    'C' : [0.1, 1, 10, 100],
    'solver' : ['liblinear']
}

grid_search = GridSearchCV(logreg, parameter_grid, cv=5, scoring = 'accuracy')
grid_search.fit(x_train, y_train)
grid_search.best_params_

## Concluding the Logistic Regression
### used VIF to identify and remove features with high multicollinearity to ensure model stability
### logistic regression machine learning model with l1 regularization was trained using the 'liblinear' solver
### GridSearchCV found the optimal parameter values for penalty, regularization strength and solver 
### Model achieved mean cross-validation score of 0.8576, which indicates good overall performance 
### The confusion matrix and metrics show a balanced performance with high precision and recall for both classes
### Precision class 1 = TP / TP + FP = 548 / 548+108 = 0.835
### Recall class 1 = TP / TP + FN = 548 / 548+89 = 0.861
### Precision class 0 = TN / TN + FN = 543 / 543+89 = 0.859
### Recall class 0 = TN / TN + FP = 543 / 543+108 = 0.834
### ROCAUC score is 0.935
### The high ROC-AUC score and balanced precision and recall metrics indicate that the model is effective at distinguishing between exoplanets discovered by transits and those discovered by other means
### The confusion matrix revealed that the model has a relatively low number of false positives and false negatives, highlighting the model's reliability